In [2]:
# Add parent directory to path so we can import project modules.
import os
import sys
from pathlib import Path

# Get parent directory (project root)
project_root = Path.cwd().parent
sys.path.append(str(project_root))
os.chdir(project_root)
print(f"Project root directory: {project_root}")

Project root directory: /Users/giandomenico/Documents/SAPIENZA/Python Scripts/rockfall_risk_roads


In [3]:
# Import standard libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import logging
import warnings
import configparser

# Import project modules
from src.utils.io_utils import read_vector, write_vector, read_raster
from src.utils import io_utils
from src.utils.geo_utils import extract_zonal_statistics, buffer_road_segments
from src.hazard.hazard_assessment import HazardAssessment
from src.road.classification import classify_road_segments_by_runout, merge_classified_segments

# Configure logging
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('hazard_assessment_testing')

# Set plot parameters
plt.rcParams['figure.figsize'] = (12, 8)

# Suppress warnings
warnings.filterwarnings('ignore', category=UserWarning)

print("Setup complete.")

Setup complete.


In [4]:
# Create HazardAssessment from config file
config_file = "config.ini"

# Check if config file exists
if not os.path.exists(config_file):
    logging.error(f"Configuration file not found: {config_file}")

# Load configuration
config = configparser.ConfigParser()
config.read(config_file)

['config.ini']

In [5]:
try:
    hazard_assessment = HazardAssessment.from_config(config_file, io_utils_module=io_utils)
except Exception as e:
    logging.error(f"Error creating HazardAssessment from config: {e}")

2025-05-06 19:05:40,458 - src.hazard.parameter_extraction - INFO - Initialized HazardParameterExtraction with buffer_distance=5.0
2025-05-06 19:05:40,461 - src.hazard.hazard_assessment - INFO - Hazard Assessment initialized
2025-05-06 19:05:40,463 - src.hazard.hazard_assessment - INFO - Loading runout extent raster from data/input/runout_total_extent_AOI.tif
2025-05-06 19:05:40,577 - src.hazard.runout_analysis - INFO - Converted runout raster to 1359 polygons
2025-05-06 19:05:40,577 - src.hazard.hazard_assessment - INFO - Runout raster updated
2025-05-06 19:05:40,578 - src.hazard.hazard_assessment - INFO - Successfully loaded runout extent raster
2025-05-06 19:05:40,578 - src.hazard.hazard_assessment - INFO - Loading susceptibility raster from data/input/runout_susceptibility_3004_AOI_fillednodata.tif
2025-05-06 19:05:40,604 - src.hazard.parameter_extraction - INFO - Added parameter raster: susceptibility with shape (1331, 1255)
2025-05-06 19:05:40,604 - src.hazard.parameter_extraction

In [6]:
# Load Road Segments (already splitted)
road_segments_path = "data/intermediate/road_segments.gpkg"
road_segments = read_vector(road_segments_path,
                            crs=config['PARAMETERS'].get('crs')
                            )
# Check for segment_id column
id_column = None
for col in ['segment_id', 'id', 'ID']:
    if col in road_segments.columns:
        id_column = col
        break

In [7]:
runout_segments, attention_segments, safe_segments = classify_road_segments_by_runout(
            road_segments,
            hazard_assessment.runout_raster,
            runout_value=hazard_assessment.runout_value,
            slope_units=hazard_assessment.slope_units,
            source_areas=hazard_assessment.source_areas,
            clumps=hazard_assessment.rockfall_clumps
            )

2025-05-06 19:05:41,074 - src.road.classification - INFO - Converting runout raster to polygons
2025-05-06 19:05:41,155 - src.road.classification - INFO - Created 1359 runout polygons
2025-05-06 19:05:41,177 - src.road.classification - INFO - Identified 188 segments in runout zone
2025-05-06 19:05:41,177 - src.road.classification - INFO - Identifying Area of Attention segments
2025-05-06 19:05:41,188 - src.road.classification - INFO - Identified 113 slope units containing source areas
2025-05-06 19:05:41,222 - src.road.classification - INFO - Identified 214 slope units containing clumps
2025-05-06 19:05:41,248 - src.road.classification - INFO - Identified 375 segments in Area of Attention
2025-05-06 19:05:41,249 - src.road.classification - INFO - Identified 31 segments Not at Risk


In [8]:
segments_with_parameters = hazard_assessment.extract_hazard_parameters(runout_segments)

2025-05-06 19:05:41,253 - src.hazard.hazard_assessment - INFO - Extracting hazard parameters from rasters
2025-05-06 19:05:41,254 - src.hazard.parameter_extraction - INFO - Road segments CRS: EPSG:3004
2025-05-06 19:05:41,254 - src.hazard.parameter_extraction - INFO - Number of road segments: 188
2025-05-06 19:05:41,254 - src.hazard.parameter_extraction - INFO - Creating 5.0m buffer zones around 188 road segments
2025-05-06 19:05:41,258 - src.hazard.parameter_extraction - INFO - Extracting ['min', 'mean', 'max', 'std'] statistics for parameter: susceptibility
2025-05-06 19:05:41,259 - src.hazard.parameter_extraction - INFO - Raster shape for susceptibility: (1331, 1255)
2025-05-06 19:05:41,301 - src.utils.geo_utils - INFO - Original raster shape for susceptibility: (1331, 1255)
2025-05-06 19:05:41,302 - src.utils.geo_utils - INFO - Raster transform: | 5.00, 0.00, 2360946.41|
| 0.00,-5.00, 4644292.50|
| 0.00, 0.00, 1.00|, NoData value: None
2025-05-06 19:05:41,683 - src.hazard.parameter

In [9]:
runout_segments_with_hazard = hazard_assessment.classify_hazard(segments_with_parameters)

2025-05-06 19:05:42,463 - src.hazard.hazard_assessment - INFO - Classifying hazard parameters


In [10]:
runout_segments_with_hazard

,OBJECTID,tr_str_sed,tr_str_fon,tr_str_liv,tr_str_sta,tr_str_ty,scril,tr_str_med,tr_str_nom,tr_str_ped,...,energy_mean,energy_max,energy_std,susceptibility_class,susceptibility_value,velocity_class,velocity_value,energy_class,energy_value,hazard_class
1,697.0,01,01,02,01,01,None,02,UNK,02,...,0.000000,0.000000,0.000000e+00,Very High,5,Very Low,1,Very Low,1,Moderate
6,3323.0,01,01,02,01,01,None,02,UNK,02,...,8.180750,8.587225,5.748425e-01,Very High,5,Low,2,Very Low,1,Moderate
494,103216.0,01,01,02,01,01,None,02,VIA SALVO D` ACQUISTO,02,...,9.819462,14.735599,2.881661e+00,Very High,5,Low,2,Very Low,1,Moderate
505,103973.0,01,01,02,01,01,None,02,VIA SALVO D` ACQUISTO,02,...,8.100492,17.174450,5.900538e+00,High,4,Low,2,Very Low,1,Moderate
525,106749.0,01,01,02,01,01,None,02,VIA SALVO D` ACQUISTO,02,...,8.180750,8.587225,5.748425e-01,Very High,5,Low,2,Very Low,1,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,143011.0,02,01,02,01,01,None,02,SP 45A,02,...,8.587225,8.587225,0.000000e+00,Very High,5,Low,2,Very Low,1,Moderate
512,104979.0,01,01,02,01,01,None,02,UNK,02,...,15.509280,15.509280,0.000000e+00,Low,2,Low,2,Very Low,1,Low
527,106816.0,01,01,02,01,01,None,02,UNK,02,...,0.000000,0.000000,0.000000e+00,Very High,5,Very Low,1,Very Low,1,Moderate
551,108761.0,01,01,02,01,01,None,02,SP40B,02,...,7.144927,8.587225,1.442297e+00,Moderate,3,Low,2,Very Low,1,Low
